#### First things first
Let's see if we can connect to the service we have deployed through AutoML...

In [6]:
from azureml.core import Workspace
from azureml.core import Webservice
from azureml.core import Model

#Load workspace
ws = Workspace.from_config(path=".file-path/ws_config.json") #I have no idea where that path value has come from, but it works

#Load services available in this workspace
services = Webservice.list(ws)
classification_service = services[0]
print(classification_service.scoring_uri)
#print(classification_service.get_keys()) #Authentication is disabled by default, so this will crash and burn!

http://1a88cd84-f73c-44fb-bd71-a2b0e8ccd881.westus.azurecontainer.io/score


Okay! That's a decent start. Now I'm guessing I can use that scoring URL to do my test predictions without having to load the model into here?

Let's start by configuring a simple JSON object to send up in the request. I sure hope there's an easy way to build this request object from a CSV / dataframe... guess we'll soon find out!  
Now which features did I train on again...?

In [3]:
data = {"data":
        [
            [
                'Fremont',
                'California',
                'United States',
                '1/05/1945',
                'Bachelors',
                'Management',
                'F',
                'S',
                0,
                2,
                0,
                5,
                86931   
            ]          
        ]
        }

That'll do for now... let's see if we can call it!

In [4]:
import requests
import json

def get_response(req_data):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(classification_service.scoring_uri, req_data, headers=headers)
    return response

request_data = json.dumps(data)
response = get_response(request_data)
response_data = json.loads(json.loads(response.text)) #Bit odd that we need to double up on the loads function, but it comes back as a string otherwise
print(response_data['result'])

['0']


Yusss.... okay we have a response! That's kind of awesome!  
I don't know whether that zero is the classification result, or just a weird response code though. Better try again with some more data :)  
We have been supplied with a set of 500 observations that the model has not seen before. I have already mangled the CSV so that the feature columns we ignored during training have been removed.

In [5]:
#Load rows from test set in CSV. Convert to lists, serialise as JSON, and send it off.
import pandas as pd

test_values = pd.read_csv('AW_Classification_Test.csv')
test_data = test_values.values.tolist()

data = {"data": test_data }
request_data = json.dumps(data)
response = get_response(request_data)
response_data = json.loads(json.loads(response.text)) #Bit odd that we need to double up on the loads function, but it comes back as a string otherwise
print(response_data['result'])

response_data_frame = pd.DataFrame(response_data['result'])
response_data_frame.shape
pd.DataFrame(response_data['result']).to_csv("AW_Classification_Results.csv", index=None)

['0', '1', '0', '0', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',

That definitely looks like a set of classification results :)  
What do we get when we feed it into the grading sheet?  
![alt text](result.PNG "Grading sheet accuracy result")

Holy heck! 80% accuracy, with what must have been about 2 hours work all up (most of that sitting around waiting for training and deployments to complete).  
Next step? Is to follow the manual process and see if I can at least match that myself :D  
Another thing I would like to try is to load the model and work against it directly in a notebook - that way I'm not dependent on my expensive Azure services to be running to use it.